In this notebook we will load and use a token classifier language model that assigns labels to some tokens in a text. Learn more about the Question Answering task <a href="https://huggingface.co/tasks/token-classification" target="_blank" rel="noopener">here</a>.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

Prior to using this notebook one needs to complete the follow steps:
1. [Create the database schema](../setup_db.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

In [9]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
from dataclasses import dataclass

@dataclass
class SandboxConfig:
    EXTERNAL_HOST_NAME = "192.168.124.93"
    HOST_PORT = "8888"

    @property
    def EXTERNAL_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.HOST_PORT}"""

    USER = "sys"
    PASSWORD = "exasol"
    BUCKETFS_PORT = "6666"
    BUCKETFS_USER = "w"
    BUCKETFS_PASSWORD = "write"
    BUCKETFS_USE_HTTPS = False
    BUCKETFS_SERVICE = "bfsdefault"
    BUCKETFS_BUCKET = "default"

    @property
    def EXTERNAL_BUCKETFS_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.BUCKETFS_PORT}"""

    @property
    def BUCKETFS_URL_PREFIX(self):
        return "https://" if self.BUCKETFS_USE_HTTPS else "http://"

    @property
    def BUCKETFS_PATH(self):
        # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
        return f"/buckets/{self.BUCKETFS_SERVICE}/{self.BUCKETFS_BUCKET}"

    SCRIPT_LANGUAGE_NAME = "PYTHON3_60"
    UDF_FLAVOR = "python3-ds-EXASOL-6.0.0"
    UDF_RELEASE= "20190116"
    UDF_CLIENT = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
    SCHEMA = "IDA"

    @property
    def SCRIPT_LANGUAGES(self):
        return f"""{self.SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}?lang=python#buckets/{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}/exaudf/{self.UDF_CLIENT}""";

    @property
    def connection_params(self):
        return {"dns": self.EXTERNAL_HOST, "user": self.USER, "password": self.PASSWORD, "compression": True}

    @property
    def params(self):
        return {
            "script_languages": self.SCRIPT_LANGUAGES,
            "script_language_name": self.SCRIPT_LANGUAGE_NAME,
            "schema": self.SCHEMA,
            "BUCKETFS_PORT": self.BUCKETFS_PORT,
            "BUCKETFS_USER": self.BUCKETFS_USER,
            "BUCKETFS_PASSWORD": self.BUCKETFS_PASSWORD,
            "BUCKETFS_USE_HTTPS": self.BUCKETFS_USE_HTTPS,
            "BUCKETFS_BUCKET": self.BUCKETFS_BUCKET,
            "BUCKETFS_PATH": self.BUCKETFS_PATH
        }

    # Name of the BucketFS connection
    BFS_CONN = 'MyBFSConn'

    # Name of a sub-directory of the bucket root
    BFS_DIR = 'my_storage'

    # We will store all models in this sub-directory at BucketFS
    TE_MODELS_DIR = 'models'
    
    # We will save cached model in this sub-directory relative to the current directory on the local machine.
    TE_MODELS_CACHE_DIR = 'models_cache'

    @property
    def WEBSOCKET_URL(self):
        return f"exa+websocket://{self.USER}:{self.PASSWORD}@{self.EXTERNAL_HOST}/{self.SCHEMA}?SSLCertificate=SSL_VERIFY_NONE"

conf = SandboxConfig()

First let's bring up the JupySQL and connect to the database via the SQLAlchemy. Please refer to the documentation in the <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using Exasol SQLAlchemy driver.

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(conf.WEBSOCKET_URL)

%load_ext sql
%sql engine

Now we will download a model from the Huggingface Hub and put into the BucketFS.

There are two ways of doing this.
1. Using the `TE_MODEL_DOWNLOADER_UDF` UDF.
2. Downloading a model to a local drive and subsequently uploading in into the BucketFS using a CLI.

The first method requires the database machine to have internet access. Here we assume this condition is met. Otherwise please refer to another notebook where the second method is demonstrated.

To demonstrate the token classification task we will use an [English Named Entity Recognition model](https://huggingface.co/sschet/biomedical-ner-all), trained on Maccrobat to recognize the bio-medical entities (107 entities) from a given text corpus (case reports etc.).

This is a public model, therefore the last parameter - the name of the Huggingface token connection - can be an empty string.

Please note that loading a model, especially a big one, may take considerable time. At the time of writing we do not have any means to check the completion of this process. Notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for few moments after that, before querying the model.

In [11]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'sschet/biomedical-ner-all'

In [ ]:
%%sql
SELECT TE_MODEL_DOWNLOADER_UDF(
    '{{MODEL_NAME}}',
    '{{conf.TE_MODELS_DIR}}',
    '{{conf.BFS_CONN}}',
    ''
)

In [18]:
# We will display all model output
%config SqlMagic.displaylimit = 0

In [16]:
MY_TEXT = """
A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring
intubation and ventilatory support out of hospital. She denied preceding symptoms of chest discomfort,
palpitations, syncope or infection. The patient was afebrile and normotensive, with a sinus tachycardia
of 140 beats/min.
"""

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_TOKEN_CLASSIFICATION_UDF(
        NULL,
        '{{conf.BFS_CONN}}',
        NULL,
        '{{conf.TE_MODELS_DIR}}',
        '{{MODEL_NAME}}',
        '{{MY_TEXT}}',
        NULL
    )
)
SELECT start_pos, end_pos, word, entity, error_message FROM MODEL_OUTPUT ORDER BY start_pos, end_pos